<a href="https://colab.research.google.com/github/waltechel/tensorflow-ml-nlp-tf2/blob/master/6.CHATBOT/chapter06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>